In [1]:
%pip install accelerate
%pip install sacrebleu

import torch
from transformers import pipeline
from tqdm import tqdm
import json
from sacrebleu.metrics import BLEU



# save the challenge set in json format as lists

references = []
sources = []

with open("Challenge_set-v2hA.json", 'r', encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        references.append(data["reference"])
        sources.append(data["source"])

# remove all the punctuations from the references and sources

import string

def remove_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

references_cleaned = [remove_punctuations(ref) for ref in references]
sources_cleaned = [remove_punctuations(src) for src in sources]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
# mistral based model

device = device = torch.device("cuda")
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [3]:
# translate the sources to french

translated_sources_llm = []

for src in tqdm(sources_cleaned):
    message = [
        {
            "role" : "user",
            "content" : f"translate the following text to french: '{src}'"
        }
    ]

    prompt = pipe.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

    output = pipe(prompt, truncation=True, max_new_tokens=256, max_length=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, num_return_sequences=1)
    translated_sources_llm.append(output[0]["generated_text"])


  0%|          | 0/108 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  1%|          | 1/108 [00:09<16:20,  9.16s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  2%|▏         | 2/108 [00:11<09:19,  5.28s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more inform

In [4]:
translated_sources_llm

['<|user|>\ntranslate the following text to french: \'The repeated calls from his mother should have alerted us\'</s>\n<|assistant|>\nLes répétées appels de sa mère devraient nous avoir mis en garde" en français.\n\nUne traduction plus littérale serait : "Les appels répétés de sa mère nous auraient dû faire prévenir"',
 '<|user|>\ntranslate the following text to french: \'The sudden noise in the upper rooms should have alerted us\'</s>\n<|assistant|>\n"Le bruit soudain dans les chambres hautes nous devrait avoir mis en garde"',
 '<|user|>\ntranslate the following text to french: \'Their repeated failures to report the problem should have alerted us\'</s>\n<|assistant|>\n"Ses répétées négligences à ne pas signaler le problème devraient nous avoir mis en garde"',
 '<|user|>\ntranslate the following text to french: \'She asked her brother not to be arrogant\'</s>\n<|assistant|>\n"Elle a demandé à son frère de ne pas être arrogant"\n\nIn French, the verb "to be" is conjugated differently d

In [5]:
# save the translated sentences by the model in a list

translated_llm = []

for text in translated_sources_llm:
    extracted_sentence = text.split("\n")[3]
    translated_llm.append(extracted_sentence.strip())

translated_llm

['Les répétées appels de sa mère devraient nous avoir mis en garde" en français.',
 '"Le bruit soudain dans les chambres hautes nous devrait avoir mis en garde"',
 '"Ses répétées négligences à ne pas signaler le problème devraient nous avoir mis en garde"',
 '"Elle a demandé à son frère de ne pas être arrogant"',
 '"Elle a promis à son frère de ne pas être arrogante"',
 '"Elle a promis à son médecin de rester active après avoir pris sa retraite"',
 '"Ma mère a promis à mon père d\'être plus prudente sur la route"',
 '"La femme était très grande et extrêmement forte."',
 '"Les leurs politiques étaient plus ignorants que stupides"',
 'Nous criâmes un insulte et nous sommes partis brusquement.',
 '"Le chat et le chien devraient être surveillés"',
 '"Mon père et mon frère seront heureux demain"',
 '"Mon livre et mon crayon peuvent être volés"',
 '"Le vache et le poulet doivent être alimentés"',
 '"Mienne mère et ma sœur seront heureuses demain"',
 '"Mes chaussures et mes souliers seront tr

In [6]:
# clean the translated sentences removing punctuations and words between parentheses.

import re

def remove_punctuation(sentence):
    sentence_without_punctuation = re.sub(r'[^\w\s\']', '', sentence)
    return sentence_without_punctuation

def filter_sentences(sentences):
    filtered_sentences = []
    for sentence in sentences:
        if '(' not in sentence and ')' not in sentence:
            filtered_sentences.append(remove_punctuation(sentence))
    return filtered_sentences

filtered_sentences = filter_sentences(translated_llm)

filtered_sentences

['Les répétées appels de sa mère devraient nous avoir mis en garde en français',
 'Le bruit soudain dans les chambres hautes nous devrait avoir mis en garde',
 'Ses répétées négligences à ne pas signaler le problème devraient nous avoir mis en garde',
 'Elle a demandé à son frère de ne pas être arrogant',
 'Elle a promis à son frère de ne pas être arrogante',
 'Elle a promis à son médecin de rester active après avoir pris sa retraite',
 "Ma mère a promis à mon père d'être plus prudente sur la route",
 'La femme était très grande et extrêmement forte',
 'Les leurs politiques étaient plus ignorants que stupides',
 'Nous criâmes un insulte et nous sommes partis brusquement',
 'Le chat et le chien devraient être surveillés',
 'Mon père et mon frère seront heureux demain',
 'Mon livre et mon crayon peuvent être volés',
 'Le vache et le poulet doivent être alimentés',
 'Mienne mère et ma sœur seront heureuses demain',
 'Mes chaussures et mes souliers seront trouvés',
 'Le chien et le bœuf so

In [7]:
# bleu score of the mistral based model

bleu = BLEU()
bleu_score_llm = bleu.corpus_score(filtered_sentences, references_cleaned)
bleu_score_llm.score

0.41800406854321887